In [1]:
import duckdb
import sqlite3
from pathlib import Path
import pandas as pd 
import re 

In [2]:
class DBConn(object):
    def __init__(self, file_db):
        """Support only DuckDB and SQLite
        """
        if not Path(file_db).exists():
            raise Exception(f"Database file not found: {file_db}")
        if file_db.endswith("duckdb"):
            self.conn = duckdb.connect(file_db)
        else:
            self.conn = sqlite3.connect(file_db)

    def __enter__(self):
        return self.conn

    def __exit__(self, type, value, traceback):
        self.conn.close()

In [3]:
FILE_DB = "db/cs-faculty-20230429.duckdb"

def alter_table_add_column(table_name, col_name, col_type = "VARCHAR", file_db=FILE_DB):
    df_1, df_2, err_msg = None, None, ""
    with DBConn(file_db) as _conn:
        try:
            select_sql = f"""
                select {col_name} from {table_name};
            """
            df_1 = _conn.execute(select_sql).df()
        except Exception as ex:
            err_msg = str(ex)

            if re.search(r"column(.*)not found", err_msg):
                alter_sql = f"""
                    ALTER TABLE {table_name} add column {col_name} {col_type};
                """
                df_2 = _conn.execute(alter_sql).df()
    return df_1, df_2, err_msg

def alter_table_drop_column(table_name, col_name, file_db=FILE_DB):
    df, err_msg = None, ""
    with DBConn(file_db) as _conn:
        try:
            alter_sql = f"""
                ALTER TABLE {table_name} drop {col_name};
            """
            df = _conn.execute(alter_sql).df()
        except Exception as ex:
            err_msg = str(ex)
    return df, err_msg

# drop column

In [8]:
for table_name in ["g_person", "g_work"]:
    col_name = "award"
    alter_table_drop_column(table_name, col_name)

# add column

In [5]:
for table_name in ["g_person", "g_work"]:
    col_name = "award"
    df_1, df_2, err_msg = alter_table_add_column(table_name, col_name)
    assert df_1 is None 
    assert df_2 is not None 
    assert err_msg != ""

# verify

In [9]:
col_name = "award"
with DBConn(file_db=FILE_DB) as _conn:
    for table_name in ["g_person", "g_work"]:
        select_sql = f"""
            select {col_name} from {table_name};
        """
        df = _conn.execute(select_sql).df()
        assert df.shape[0] > 0 

BinderException: Binder Error: Referenced column "award" not found in FROM clause!
Candidate bindings: "g_person.id"
LINE 2:             select award from g_person;
                           ^